In [ ]:
import numpy as np
import pandas as pd
from utils import get_data, get_preds, get_table, plot_rmse, other_stats, add_intervals_to_test, plot_interval_check, plot_interval_sizes, plot_finish_groups
np.random.seed(2024)

size = 500
save_val = False
train_yr, test_yr = [2022], [2023]
train_bos, test_bos = get_data(racename="bos", size_train=size, size_test=size, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_nyc, test_nyc = get_data(racename="nyc", size_train=size, size_test=size, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_chi, test_chi = get_data(racename="chi", size_train=size, size_test=size, train_lis=train_yr, test_lis=test_yr, save=save_val)
data = {"bos": (train_bos, test_bos), "nyc": (train_nyc, test_nyc), "chi": (train_chi, test_chi)}
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [24]:
race = "chi"
train, test = data[race]

model_info = [
    # ("rstan2d", "stan_results/params_bos0d.csv", ["total_pace", "curr_pace", "prop"], ["beta[1]", "beta[2]", "beta[3]"], True),
    ("model1", f"stan_results/model1/params_{race}.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    ("model2", f"stan_results/model2/params_{race}.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
]

# models = {name: pd.read_csv(path) for (name, path, feats, betas, pleft) in model_info}
mpreds = {name: get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name)
           for (name, path, feats, betas, pleft) in model_info}

test2 = get_table(test, mpreds)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,extrap,model1,model2
0,78846,5K,3.196931,3.196931,3.254280,-,-,2023,0.118497,0.881503,0,0.378828,,3.876600,20.544604,21.174255
1,75624,5K,3.380663,3.380663,3.419645,-,-,2023,0.118497,0.881503,0,0.400600,,2.371350,17.221993,17.824094
2,82302,5K,3.006615,3.006615,3.100066,-,-,2023,0.118497,0.881503,0,0.356276,,7.050950,25.978339,26.638927
3,113742,5K,1.909126,1.909126,1.791720,-,-,2023,0.118497,0.881503,0,0.226227,,-24.137650,24.964754,25.851970
4,102201,5K,2.551020,2.551020,2.384976,-,-,2023,0.118497,0.881503,0,0.302289,,-19.192667,7.455911,8.201491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,103427,40K,2.312673,2.332906,2.340526,-,-,2023,0.947980,0.052020,0,2.192367,,0.981446,1.978654,-0.203266
3996,90674,40K,2.655337,2.803280,2.797335,-,-,2023,0.947980,0.052020,0,2.517206,,-0.533144,0.177577,-0.411394
3997,100447,40K,2.414293,2.439917,2.447790,-,-,2023,0.947980,0.052020,0,2.288700,,0.927013,1.844687,-0.064752
3998,81003,40K,2.682403,3.193358,3.152645,-,-,2023,0.947980,0.052020,0,2.542864,,-2.843929,-2.283352,-0.912795


In [25]:
# s_data =pd.DataFrame([[4, 0.2, .5], [3, 0.5, .75]], columns=["total_pace", "prop", "propleft"])
# get_preds(s_data, models['rstan2c'], feats_lis = ["total_pace", "prop"], beta_lis = ["beta[1]", "beta[2]"], full=True)

In [26]:
labels = ["model1", "model2", "extrap"]
rmse_table = plot_rmse(test2, labels, save_name=f"{race}")
for lbl in labels[:-1]:
    rmse_table[f"pcnt_{lbl}"] = 1 - (rmse_table[lbl] / rmse_table["extrap"])
rmse_table

File saved: analysis/chi_rmse.png


,model1,model2,extrap,pcnt_model1,pcnt_model2
dist,,,,,
5K,21.752018,21.969233,26.999029,0.194341,0.186295
10K,18.234489,18.242803,23.767815,0.232808,0.232458
15K,15.435281,15.226083,22.205943,0.304903,0.314324
20K,13.561631,13.310564,19.884137,0.317967,0.330594
25K,10.993102,10.580330,15.853004,0.306560,0.332598
30K,8.072705,7.499398,11.234051,0.281407,0.332440
35K,5.030738,4.297344,6.568626,0.234126,0.345777
40K,1.581717,1.650201,1.577976,-0.002371,-0.045771


In [27]:
plot_finish_groups(test2, label_pair=["extrap", "model2"], num=4, overall=True, save_name=race)

File saved: analysis/chi_rmse_groups.png


In [ ]:
#group.plot(label=table_group.columns, style=styles, linewidth=2, grid=True, alpha=0.8, color=colors)
# group.swaplevel(0,1).stack(0)#.loc["F"]

In [30]:
other_stats(test2[labels], test2["finish"])

,model1,model2,extrap
0,13.424044,13.293920,18.046807
1,0.949634,0.950605,0.908972


In [ ]:
mpreds2 = {name: (42195 / 60) / get_preds(
                test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name, full=True
            ) for (name, path, feats, betas, pleft) in model_info}
# model_preds2["new1"] = (42195 / 60) / get_preds(test, pd.read_csv("stan_results/params_bos1.csv"), feats_lis=['total_pace', 'curr_pace', 'prop'], beta_lis=['beta.1', 'beta.2', 'beta.3'], propleft=True, full=True)

In [33]:
pred_names = labels[:2]
intervals_tbl = add_intervals_to_test(test2, mpreds2, pred_names)
i_check = plot_interval_check(intervals_tbl, pred_names, save_name=f"{race}")
i_sizes = plot_interval_sizes(intervals_tbl, pred_names, save_name=f"{race}")

File saved: analysis/chi_interval_check.png
File saved: analysis/chi_interval_sizes.png


<Figure size 4000x4000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>